In [41]:
import bidding

import jack_to_deal_auction as jack_format

from lstm_bidder import Bidder

In [5]:
jack_format.convert_deal_str('W:KJ43.T865.K3.K98 Q8.A974.62.AJ762 T75.J3.Q985.QT43 A962.KQ2.AJT74.5')

'SQ8 HA974 D62 CAJ762  ST75 HJ3 DQ985 CQT43  SA962 HKQ2 DAJT74 C5  SKJ43 HT865 DK3 CK98'

In [18]:
jack_format.convert_auction_str('PP')

'P'

In [8]:
jack_format.parse_dealer_vuln('N -')

('N', 'None')

In [77]:
def to_auction_list(auction_str):
    i = 0
    result = []
    while i < len(auction_str):
        result.append(auction_str[i:i+2])
        i += 2
    return result
        

In [78]:
to_auction_list('')

[]

In [81]:
def parse_line(line):
    hand_metadata = line.strip().split('#:')
    deal_str = jack_format.convert_deal_str(hand_metadata[0])
    metadata_parts = hand_metadata[1].split()
    table = int(metadata_parts[2][2:])
    dealer = metadata_parts[3][2:]
    vuln = metadata_parts[4][2:]
    jack_auction_str = ' '.join(to_auction_list(metadata_parts[5][2:]))
    auction_str = '%s %s %s' % (dealer, jack_format.jack_vuln_lookup[vuln], jack_format.convert_auction_str(jack_auction_str))
    
    return DealData.from_deal_auction_string(deal_str, auction_str)

In [82]:
dd = parse_line('W:KJ43.T865.K3.K98 Q8.A974.62.AJ762 T75.J3.Q985.QT43 A962.KQ2.AJT74.5 #:0001 C:BW5C T:1 D:N V:- A:1C')

1C


In [83]:
dd.auction

['1C']

In [48]:
len(dd.auction) % 4

1

In [44]:
bidder = Bidder('bw5c', './bw5c_model/bw5c-500000')

INFO:tensorflow:Restoring parameters from ./bw5c_model/bw5c-500000


In [45]:
dd.dealer, dd.vuln_ns, dd.vuln_ew

(1, True, False)

In [63]:
bidder.next_bid(dd, ['1C', 'PASS', '1D', 'PASS', '1H', 'PASS', '1S', 'PASS', '1N', 'PASS', '3N', 'PASS', 'PASS', 'PASS'])

'PAD_END'

In [64]:
def bid_to_jack_format(bid_str):
    lookup = {
        'PASS': 'PP',
        'X': 'DD',
        'XX': 'RR'
    }
    if bid_str in lookup:
        return lookup[bid_str]
    else:
        return bid_str